In [1]:
from Load_Data import get_data

In [2]:
database, fecha = get_data()

Se están cargando los archivos de la carpeta, c:\Users\andre\OneDrive\Escritorio\DIP\01_Reporte de Ejecución Presupuestal\03_Código_Limpieza_Tablero\Scripts\..\Archivos Base\Reportes SIIF Vigencia\23-07-31


In [8]:
database.keys()

dict_keys(['CDP', 'CDP_FIP', 'RP', 'RP_FIP', 'Oblig', 'Oblig_FIP', 'OP', 'OP_FIP', 'Ejecución_Presupuestal_Agregada', 'Oblig_Reservas', 'Oblig_FIP_Reservas', 'OP_Reservas', 'OP_FIP_Reservas', 'RP_Reservas', 'RP_FIP_Reservas', 'CDP_22', 'CDP_FIP_22', 'RP_22', 'RP_FIP_22', 'Oblig_22', 'Oblig_FIP_22', 'OP_22', 'OP_FIP_22', 'Ejecución_Presupuestal_Agregada_22', 'RP_Reservas_22', 'RP_FIP_Reservas_22', 'Oblig_Reservas_22', 'Oblig_FIP_Reservas_22', 'OP_Reservas_22', 'OP_FIP_Reservas_22', 'CDP_21', 'CDP_FIP_21', 'RP_21', 'RP_FIP_21', 'Oblig_21', 'Oblig_FIP_21', 'OP_21', 'OP_FIP_21', 'Ejecución_Presupuestal_Agregada_21', 'RP_Reservas_21', 'RP_FIP_Reservas_21', 'Oblig_Reservas_21', 'Oblig_FIP_Reservas_21', 'OP_Reservas_21', 'OP_FIP_Reservas_21'])

In [9]:
fecha

'23-07-31'

In [11]:
from datetime import datetime
fecha = datetime.strptime(fecha, "%y-%m-%d")

In [13]:
fecha.year

2023